In [ ]:
import mesa
import numpy as np
import mesa
import random
import math
import modules as modules
import model as model
import matplotlib.pyplot as plt
import pandas as pd
import optuna

In [ ]:
def compute_entropy(model: mesa.Model, loc: tuple, agent_type):
    similar = 0
    num_neighbors = 0

    for neighbor in model.grid.iter_neighbors(loc, moore=True, radius=model.radius):
        num_neighbors += 1
        if neighbor.type == agent_type:
            similar += 1

    if num_neighbors == 0:
        return 0  # No neighbors means zero entropy

    proportion_similar = similar / num_neighbors
    proportion_different = 1 - proportion_similar

    if proportion_similar == 0 or proportion_different == 0:
        return 0  # If all neighbors are similar or all are different, entropy is zero

    entropy = - (proportion_similar * math.log2(proportion_similar) + proportion_different * math.log2(proportion_different))
    return entropy

# Added Modules

In [ ]:
def utility_func(model: mesa.Model, agent: mesa.Agent, agent_loc: tuple, property_loc: tuple) -> float:
    
    theta = model.get_theta(agent_loc, model.grid.get_property(property_loc))

    desirability = model.desirability_layer(property_loc)

    alpha = model.alpha

    budget = agent.budget
    
    price = model.price_func(property_loc)
    
    return theta**alpha*desirability**(1-alpha)*((budget-price)/budget)

In [ ]:
NO_NEIGHBORS_THETA = 0.5

class SchellingAgent(mesa.Agent):
    """
    Schelling segregation agent
    """

    def __init__(self, unique_id, model, agent_type, budget):
        """
        Create a new Schelling agent.

        Args:
           unique_id: Unique identifier for the agent.
           agent_type: Indicator for the agent's type (minority=1, majority=0)
           budget: Budget for the agent
        """
        super().__init__(unique_id, model)
        self.type = agent_type
        self.budget = budget
        self.utility = 0.5
        self.segregation = None
        self.move_counter = 0

    def calc_theta(self):
        # Calculate theta using the model's get_theta method
        self.segregation = modules.get_theta(self.model, self.pos, self.type)

    def step(self):
        """
        Step for agent to move
        In a step an agent will:
            1. Find available properties to move to
            2. Calculate their utility for each property
            3. If the property with the highest utility has a higher utility than the current property, move there
            4. Update the utility of the agent in their new location
        """
        # update utility
        self.utility = self.model.utility_func(self.model, self, self.pos)
        
        self.calc_theta()

        # find the available properties to move to
        available_cells = self.model.find_available_cells(self)
                
        if len(available_cells) < 0:
            return
        
        # list all utilities of available properties
        move_util = []
        for cell in available_cells:
            # store as (cell, utility) tuple
            move_util.append((cell, self.model.utility_func(self.model, self, cell)))
        
        # sort by utility
        move_util.sort(key=lambda x: x[1], reverse=True)
        
        # move if utility is higher than current
        if move_util[0][1] > self.utility:
            self.model.grid.move_agent(self, move_util[0][0])
            # update utility
            self.utility = move_util[0][1]
            self.move_counter += 1


class Schelling(mesa.Model):
    """
    Model class for the Schelling segregation model.
    """

    def __init__(
        self,
        property_value_func,
        income_func,
        desirability_func,
        utility_func,
        price_func,
        ##########
        compute_similar_neighbours,
        ##########
        height=20,
        width=20,
        homophily=0.5,
        radius=1,
        density=0.8,
        minority_pc=0.2,
        alpha=0.5,
        income_scale=1.5, # the scale by which the income is higher than the property value
        property_value_weight=0.1,
        mu_theta = 0.8,
        sigma_theta = 0.1,
        entropy = -1, # initialize entropy for a non-possible value
        seed=None
    ):
        """
        Create a new Schelling model.

        Args:
            width, height: Size of the space.
            density: Initial chance for a cell to be populated
            minority_pc: Chance for an agent to be in minority class
            homophily: Minimum number of agents of the same class needed to be happy
            radius: Search radius for checking similarity
            seed: Seed for reproducibility
            property_value: Value for the property
        """

        super().__init__(seed=seed)
        self.utility_func = utility_func
        self.price_func = price_func
        self.desirability_func = desirability_func
        self.prop_value_weight = property_value_weight
        self.height = height
        self.width = width
        self.density = density
        self.minority_pc = minority_pc
        self.homophily = homophily
        self.radius = radius
        self.alpha = alpha
        self.mu_theta = mu_theta
        self.sigma_theta = sigma_theta
        self.entropy = entropy
        
        #############
        self.compute_similar_neighbours = compute_similar_neighbours
        self.neighbor_similarity_counter = {}
        #############

        self.schedule = mesa.time.RandomActivation(self)
        self.grid = mesa.space.SingleGrid(width, height, torus=True)

        # Property Value Layer
        self.property_value_layer = property_value_func(name="property_values", width=width, height=height)
        self.grid.add_property_layer(self.property_value_layer)

        # Desirability Layer
        self.desirability_layer = mesa.space.PropertyLayer("desirability", width, height, 0.5)
        # for _, pos in self.grid.coord_iter():
        #     self.desirability_layer[pos] = 1
        self.grid.add_property_layer(self.desirability_layer)
        
        # Interested Agents Counter Layer
        self.interested_agents_layer = mesa.space.PropertyLayer("interested_agents", width, height, 0)
        # for _, pos in self.grid.coord_iter():
        #     self.interested_agents_layer[pos] = 0
        self.grid.add_property_layer(self.interested_agents_layer)
        
        # Utility Layer
        self.utility_layer = mesa.space.PropertyLayer("utility", width, height, 0.5) 
        self.grid.add_property_layer(self.utility_layer)

        #Data Collectors
        self.datacollector = mesa.DataCollector(
            agent_reporters={"Utility": "utility", "Segregation":"segregation", "Moves":"move_counter"}, model_reporters={"Desirability": self.desirability_layer.data.tolist}  # Collect the utility of each agent
        )

        # Set up agents
        for _, pos in self.grid.coord_iter():
            if self.random.random() < self.density:
                agent_type = 1 if self.random.random() < self.minority_pc else 0
                budget = income_func(scale=income_scale)
                agent = SchellingAgent(self.next_id(), self, agent_type, budget)
                self.grid.place_agent(agent, pos)
                self.schedule.add(agent)

        self.datacollector.collect(self)

    def find_available_cells(self, agent):
        available_cells = []
        for _, pos in self.grid.coord_iter():
            if self.grid.is_cell_empty(pos):
                available_cells.append(pos)        
        return available_cells

    def step(self):
        """
        Run one step of the model.
        """
        # Set the count of agents who like to move somewhere to 0 for all cells
        self.interested_agents_layer.set_cells(0)

        ########
        self.neighbor_similarity_counter.clear()
        ########

        for agent in self.schedule.agents:
            # Iterate over cells and compare utility to current location, add to interested_agents_layer if better
            for _, loc  in self.grid.coord_iter():
                utility = self.utility_func(self, agent, loc)
                
                if utility > agent.utility:
                    self.interested_agents_layer.modify_cell(loc, lambda v: v + 1)

        ###### ADDED #############
            # Compute number of agents with the same number of similar neighbours 
            similar_neighbors = self.compute_similar_neighbours(self, agent)
            if similar_neighbors not in self.neighbor_similarity_counter:
                self.neighbor_similarity_counter[similar_neighbors] = 0
            self.neighbor_similarity_counter[similar_neighbors] += 1

        # Compute total number of agents included
        total_agents = sum(self.neighbor_similarity_counter.values())

        # Compute entropy and store it 
        current_entopy = 0
        for _, p in self.neighbor_similarity_counter.items():
            if p > 0:  # To avoid domain error for log(0)
                probability = p / total_agents
                value = probability * np.log10(probability)
                current_entopy += value
        self.entropy = -current_entopy
        #############################

        # Set desirability layer to the proportion of interested agents
        num_agents = len(self.schedule.agents)
        self.desirability_layer.set_cells(
            self.desirability_func(self, prop_value_weight=self.prop_value_weight)
        )
        
        self.schedule.step()
        self.datacollector.collect(self)

In [ ]:
# OFAT
# # Create and run the model
model = model.Schelling(
     property_value_func=modules.property_value_quadrants,
     income_func=modules.income_func,
     desirability_func=modules.desirability_func,
     utility_func=modules.utility_func,
     price_func=modules.price_func,
     compute_similar_neighbours=modules.compute_similar_neighbours,
     height=20,
     width=20,
     homophily=0.5,
     radius=1,
     density=0.8,
     minority_pc=0.2,
     alpha=0.5,
     seed=42
 )

# # Run the model for a certain number of steps
for i in range(5):
     print(i)
     print(model.entropy)
     #print(model.neighbor_similarity_counter)
     model.step()

# Function to run model with inputs

In [ ]:
# Step 3: Run the model for generated samples
import model
import modules

# First define the model such that it runs for a specified number of time steps in a function 
def run_schelling_model(property_value_func,
                        income_func,
                        desirability_func,
                        utility_func,
                        price_func,
                        compute_similar_neighbours,
                        height,
                        width,
                        homophily,
                        radius,
                        density,
                        mu_theta,
                        sigma_theta,
                        minority_pc,
                        property_value_weight,
                        alpha,
                        seed,
                        num_steps):
    # Initialize the model
    model_instance = model.Schelling(
        property_value_func=property_value_func,
        income_func=income_func,
        desirability_func=desirability_func,
        utility_func=utility_func,
        price_func=price_func,
        compute_similar_neighbours=compute_similar_neighbours,
        height=height,
        width=width,
        homophily=homophily,
        radius=radius,
        density=density,
        mu_theta=mu_theta,
        sigma_theta=sigma_theta,
        minority_pc=minority_pc,
        property_value_weight= property_value_weight,
        alpha=alpha,
        seed=seed
    )

    # Run the model for the specified number of steps and collect entropy values
    entropies = []

    for i in range(num_steps):
        model_instance.step()
        entropies.append(model_instance.entropy)

    model_data = model_instance.datacollector.get_model_vars_dataframe()
    agent_data = model_instance.datacollector.get_agent_vars_dataframe()

    # Return the model instance and entropy values
    return model_instance, entropies, model_data, agent_data
  

# Example usage

In [ ]:
model_result, entropy_values, model_data, agent_data = run_schelling_model(
    property_value_func=modules.property_value_quadrants,
    income_func=modules.income_func,
    desirability_func=modules.desirability_func,
    utility_func=modules.utility_func,
    price_func=modules.price_func,
    compute_similar_neighbours=modules.compute_similar_neighbours,
    height=20,
    width=20,
    homophily=0.5,
    radius=1,
    density=0.8,
    mu_theta=0.8,
    sigma_theta = 0.1,
    property_value_weight= 0.1,
    minority_pc=0.2,
    alpha=0.5,
    seed=42,
    num_steps=5
)

# Print the entropy values
print("Entropy values at each step:", entropy_values)

print("Model data:")
print(model_data)

# OFAT Analysis

In [ ]:
# Define the model parameters
params = {
    'property_value_func': modules.property_value_quadrants,
    'income_func': modules.income_func,
    'desirability_func': modules.desirability_func,
    'utility_func': modules.utility_func,
    'price_func': modules.price_func,
    'compute_similar_neighbours': modules.compute_similar_neighbours,
    'height': 20,
    'width': 20,
    'homophily': 0.5,
    'radius': 1,
    'density': 0.8,
    'mu_theta': 0.8,
    'sigma_theta': 0.1,
    'minority_pc': 0.2,
    'property_value_weight':0.1,
    'alpha': 0.5,
    'seed': 42,
    'num_steps': 10  # Number of steps to run the model
}

# Define the ranges for each parameter
param_ranges = {
    #'homophily': np.linspace(0, 1, 10),
    #'radius': np.arange(1, 5),
    #'density': np.linspace(0.1, 1, 10),

    'mu_theta': np.linspace(0, 1, 10),
    'sigma_theta': np.linspace(0, 1, 10),
    'property_value_weight': np.linspace(0, 1, 10),
    'minority_pc': np.linspace(0.1, 0.5, 10),
    'alpha': np.linspace(0, 1, 10)
}


In [ ]:
# Record the results
results = []

# Run the model for each combination of parameter values

# OFAT analysis
for param, values in param_ranges.items():
    original_value = params[param]
    for value in values:
        # Update the parameter value
        params[param] = value
        
        # Run the model
        _, entropies, model_data, agent_data = run_schelling_model(**params)
        
        # Record the results
        avg_entropy = np.mean(entropies)
        results.append({
            'param': param,
            'value': value,
            'avg_entropy': avg_entropy
        })
    
    # Restore the original value of the parameter
    params[param] = original_value

# Convert results to a DataFrame
df_results = pd.DataFrame(results)

# Save the results to a CSV file
#df_results.to_csv('ofat_results.csv', index=False)

# Plot the results for each parameter
for param in param_ranges.keys():
    df_param = df_results[df_results['param'] == param]
    plt.plot(df_param['value'], df_param['avg_entropy'], label=param)

plt.xlabel('Parameter Value')
plt.ylabel('Average Entropy')
plt.title('OFAT Analysis')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print(np.max(df_results['avg_entropy']))

In [ ]:
# Record the results
results = []
repetitions = 5

# OFAT analysis
for param, values in param_ranges.items():
    original_value = params[param]
    for value in values:
        # Update the parameter value
        params[param] = value
        
        entropies = []
        for _ in range(repetitions):
            # Run the model
            _, entropy_values, model_data, agent_data = run_schelling_model(**params)
            entropies.extend(entropy_values)
        
        # Calculate mean and standard deviation
        avg_entropy = np.mean(entropies)
        std_entropy = np.std(entropies)
        
        # Record the results
        results.append({
            'param': param,
            'value': value,
            'avg_entropy': avg_entropy,
            'std_entropy': std_entropy
        })
    
    # Restore the original value of the parameter
    params[param] = original_value

# Convert results to a DataFrame
df_results = pd.DataFrame(results)

In [ ]:
# Plot the results for each parameter
plt.figure(figsize=(15, 8), dpi=300)
for param in param_ranges.keys():
    df_param = df_results[df_results['param'] == param]
    plt.errorbar(df_param['value'], df_param['avg_entropy'], yerr=df_param['std_entropy'], label=param, fmt='-o')

plt.xlabel('Parameter Value')
plt.ylabel('Average Entropy')
plt.title('OFAT Analysis with Mean and Standard Deviation')
plt.legend()
plt.grid(True)
plt.show()